# Consultancy report analysis
## Motorbike scrambling track
By Gillian Kennedy (B00805416)

## Introduction
Enter text about problem?

## Aim 
Analyse the potential impact of the motorbike scrambling track in the Binevenagh area.

## Objectives
1. Analyse potential impact on local residents
2. Analyse potential impact on environment
3. Analyse potential impact on tourism

## Data Provided
- NI_roads
- NI_rivers
- NI_outline
- ASSI
- AONB
- Landcover
- Gazeteer
- Settlements
- Census population
- Children 0-7
- Pointer data
- new buildings

## Preparing data for analysis

First we need to import the appropriate packages for the analysis.

In [ ]:
#use figures interactively
%matplotlib widget

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import folium
from shapely.geometry import Point, LineString, Polygon
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

plt.ion() #make plotting interactive

### loading the data

In [ ]:
outline = gpd.read_file(os.path.abspath('data_files/NI_outline.shp'))
roads = gpd.read_file(os.path.abspath('data_files/NI_roads.shp'))
settlements = gpd.read_file(os.path.abspath('data_files/settlements_poly.shp'))
gazeteer = gpd.read_file(os.path.abspath('data_files/Gazeteer.shp'))

Creating the map figure.

In [ ]:
ni_utm = ccrs.UTM(29) #create a Universal Transverse Mercator reference system to transform the data

In [ ]:
outline = outline.to_crs(ni_utm)
roads = roads.to_crs(ni_utm)
settlements = settlements.to_crs(ni_utm)
gazeteer = gazeteer.to_crs(ni_utm)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ni_utm)

Adding outline to the map

In [ ]:
# first, we just add the outline of Northern Ireland using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(outline['geometry'],
                                 ni_utm, edgecolor='k', 
                                 facecolor='w',
                                 zorder=1)
ax.add_feature(outline_feature) # add the features we've created to the map.

Setting the extent to Northern Ireland

In [ ]:
xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

fig ## re-draw the figure so you don't have to scroll back up

num_road_types = len(roads.road_class.unigue())
print(f'number of unique features: {num_road_types}')

In [ ]:
roads_feat = ShapelyFeature(roads['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='firebrick', # set the edgecolor to be firebrick
                            facecolor='firebrick', # set the facecolor to be firebrick
                            linewidth=0.2, # set the linewidth to be 0.2 pt
                            zorder=2) 
ax.add_feature(roads_feat) # add the collection of features to the map

settlements_feat = ShapelyFeature(settlements['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='gray', # set the edgecolor to be gray
                            facecolor='gray', # set the facecolor to be gray
                            linewidth=1,
                            zorder=3) # set the linewidth to be 0.2 pt
ax.add_feature(settlements_feat) # add the collection of features to the map

gazeteer_feat = ax.scatter(gazeteer.geometry.x, gazeteer.geometry.y, # first argument is the geometry
                            transform=ccrs.UTM(zone=29, southern_hemisphere=False), # second argument is the CRS
                            color='yellow',
                            s= 10,
                            zorder=4) 

fig # to show the updated figure
